#Import Libraries

In [4]:
%tensorflow_version 2.x

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pathlib

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.layers import MaxPool2D, MaxPool1D
from tensorflow.keras.layers import Dropout, BatchNormalization

from google.colab import drive

print("Module Loaded.")
print("TensorFlow Version :{}".format(tf.__version__))
print("NumPy Version :{}".format(np.__version__))
print("Matplotlib Version :{}".format(plt.matplotlib.__version__))

Module Loaded.
TensorFlow Version :2.0.0
NumPy Version :1.17.4
Matplotlib Version :3.1.2
Module Loaded.
TensorFlow Version :2.0.0
NumPy Version :1.17.4
Matplotlib Version :3.1.2


In [5]:
drive.mount('/content/drive')

data_path = pathlib.Path('/content/drive/Shared drives/scsa_2019_b/Project_Ai-KEA/data/furniture')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
class_name = np.array([item.name for item in data_path.glob('*')])
print(class_name)

['rug' 'curtain' 'table' 'bed' 'bookshelves' 'sofa' 'makeup_table' 'chair'
 'standing_lamp' 'entertainment_center']


# Prepare Data

- **Unzip Data**

In [7]:
import os

cwd = os.getcwd()
print(cwd)

!unzip /content/drive/Shared\ drives/scsa_2019_b/Project_Ai-KEA/data_resize/furniture.zip

/content
Archive:  /content/drive/Shared drives/scsa_2019_b/Project_Ai-KEA/data_resize/furniture.zip
  inflating: furniture/table/table407.jpg  
  inflating: furniture/table/table404.jpg  
  inflating: furniture/table/table409.jpg  
  inflating: furniture/table/table386.jpg  
  inflating: furniture/table/table397.jpg  
  inflating: furniture/table/table399.jpg  
  inflating: furniture/table/table385.jpg  
  inflating: furniture/table/table408.jpg  
  inflating: furniture/table/table400.jpg  
  inflating: furniture/table/table394.jpg  
  inflating: furniture/table/table387.jpg  
  inflating: furniture/table/table398.jpg  
  inflating: furniture/table/table392.jpg  
  inflating: furniture/table/table406.jpg  
  inflating: furniture/table/table405.jpg  
  inflating: furniture/table/table388.jpg  
  inflating: furniture/table/table391.jpg  
  inflating: furniture/table/table402.jpg  
  inflating: furniture/table/table389.jpg  
  inflating: furniture/table/table390.jpg  
  inflating: furnit

# Resize Image

In [0]:
# from PIL import Image

# for j in class_name[1:4]:
#     image = pathlib.Path('/content/drive/Shared drives/scsa_2019_b/Project_Ai-KEA/data/furniture/'+j)
#     image = list(image.glob('*.jpg'))
#     for i in range(len(image)):
#         im = Image.open(str(image[i])).convert('RGB')
#         resize_image = im.resize((im.size[0]//2,im.size[1]//2))
#         resize_image.save('/content/drive/Shared drives/scsa_2019_b/Project_Ai-KEA/data_resize/furniture/{0}/{1}{2:03d}.jpg'.format(j,j,i))
#         print(j, i)

In [0]:
data_path = pathlib.Path('/content/drive/Shared drives/scsa_2019_b/Project_Ai-KEA/data_resize/furniture')

#ImageGenerator

In [0]:
img_size = 112
batch_n = 128

data_path = pathlib.Path('/content/furniture')

In [0]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                #horizontal_flip=True, 
                                #vertical_flip=True,
                                #rotation_range=30,
                                #zoom_range=0.1,
                                #brightness_range=[0.8,1.2],
                                validation_split=0.2,
                                rescale=1./255)

In [12]:
train_data_generate = image_generator.flow_from_directory(
                        directory=str(data_path),
                        batch_size=batch_n,
                        shuffle=True,
                        target_size=(img_size, img_size),
                        classes = list(class_name),
                        subset='training')
test_data_generate = image_generator.flow_from_directory(
                        directory=str(data_path),
                        batch_size=batch_n,
                        shuffle=True,
                        target_size=(img_size, img_size),
                        classes = list(class_name),
                        subset='validation')

Found 3705 images belonging to 10 classes.
Found 921 images belonging to 10 classes.


# Load Prev Model

In [13]:
%%time
# format : ai-kea-{day01}-{sey}
generator, _, _ = load_models("ai-kea",500)
show_images(0.5 * generator.predict(noise_data) + 0.5)

NameError: ignored

#Load Inception

import model

In [15]:
import tensorflow.keras.applications.inception_v3 as incep

model_incep = incep.InceptionV3(
    weights="imagenet", 
    include_top=False,    # dense layer 이 후는 제외
    input_tensor=tf.keras.layers.Input(shape=(img_size, img_size, 3))
    #input_tensor=keras.layers.Input(shape=(49, 49, 3))
)

model_incep.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 112, 112, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 55, 55, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 55, 55, 32)   96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 55, 55, 32)   0           batch_normalization_94[0][0]     
_______________________________________________________________________________________

##fine tuning

freeze layers

In [16]:
# slicing which layers not to train
# for layer in model_incep.layers[:] :
#   layer.trainable = False

for layer in model_incep.layers :
  print(layer, layer.trainable)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7fb27e3e0518> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fb27e3e05c0> True
<tensorflow.python.keras.layers.normalization.BatchNormalization object at 0x7fb317625f98> True
<tensorflow.python.keras.layers.core.Activation object at 0x7fb27e3d7940> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fb27cb92908> True
<tensorflow.python.keras.layers.normalization.BatchNormalization object at 0x7fb27cbc3780> True
<tensorflow.python.keras.layers.core.Activation object at 0x7fb27cbce198> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fb27cbce1d0> True
<tensorflow.python.keras.layers.normalization.BatchNormalization object at 0x7fb27cb27f98> True
<tensorflow.python.keras.layers.core.Activation object at 0x7fb27cb39c18> True
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fb27caecf60> True
<tensorflow.python.keras.layers.convolutional.Conv2D 

make ai-kea model

In [17]:
ai_kea = tf.keras.models.Sequential()

ai_kea.add(model_incep)

# Here add new layers
ai_kea.add(Flatten())
#ai_kea.add(Dense(1024, activation='relu'))
#model_fine.add(keras.layers.Dropout(0.5))
#model_fine.add(keras.layers.LeakyReLU(alpha=0.1))
#ai_kea.add(BatchNormalization())   # 필히 사용할 것
ai_kea.add(Dense(10, activation='softmax'))
ai_kea.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 2, 2, 2048)        21802784  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                81930     
Total params: 21,884,714
Trainable params: 21,850,282
Non-trainable params: 34,432
_________________________________________________________________


#model compile & train

In [0]:
epoch_n = 100

In [0]:
from tensorflow.keras.optimizers import Adam

In [0]:
def lr_schedule(epoch):
    lr = 1e-2
    if epoch/epoch_n > 0.8:
        lr *= 1e-4
    elif epoch/epoch_n > 0.6:
        lr *= 1e-3
    elif epoch/epoch_n > 0.4:
        lr *= 1e-2
    elif epoch/epoch_n > 0.2:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [21]:
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau

opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule(0))
lrate = LearningRateScheduler(lr_schedule)
callbacks_list = [lrate]

Learning rate:  0.01


In [0]:
ai_kea.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [23]:
history = ai_kea.fit_generator(
    train_data_generate,
    epochs = epoch_n,
    validation_data = test_data_generate,
    #callbacks=callbacks_list
)

Epoch 1/100
29/29 [==============================] - 53s 2s/step - loss: 1.4829 - accuracy: 0.5682 - val_loss: 14.8169 - val_accuracy: 0.0847
Epoch 2/100
29/29 [==============================] - 42s 1s/step - loss: 0.7328 - accuracy: 0.7968 - val_loss: 8.4857 - val_accuracy: 0.1357
Epoch 3/100
29/29 [==============================] - 42s 1s/step - loss: 0.5465 - accuracy: 0.8445 - val_loss: 3.1160 - val_accuracy: 0.2313
Epoch 4/100
29/29 [==============================] - 42s 1s/step - loss: 1.0675 - accuracy: 0.7190 - val_loss: 14.2689 - val_accuracy: 0.1097
Epoch 5/100
29/29 [==============================] - 43s 1s/step - loss: 1.2213 - accuracy: 0.6524 - val_loss: 14.3338 - val_accuracy: 0.1097
Epoch 6/100
29/29 [==============================] - 42s 1s/step - loss: 0.9505 - accuracy: 0.7274 - val_loss: 14.0218 - val_accuracy: 0.1010
Epoch 7/100
29/29 [==============================] - 43s 1s/step - loss: 0.7462 - accuracy: 0.7911 - val_loss: 10.2606 - val_accuracy: 0.1857
Epoch 8/

- **model save**

In [0]:
# Prepare model saving directory.
model_type = 'VGG16'
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'ai-kea_%s_model.{epoch:03d}.h5' % model_type

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

In [0]:
history = ai_kea.fit_generator(
    train_data_generate,
    epochs = epoch_n,
    validation_data = test_data_generate
)

Epoch 1/100
29/29 [==============================] - 48s 2s/step - loss: 0.0540 - accuracy: 0.9811 - val_loss: 1.2285 - val_accuracy: 0.7850
Epoch 2/100
29/29 [==============================] - 43s 1s/step - loss: 0.0609 - accuracy: 0.9792 - val_loss: 1.2674 - val_accuracy: 0.7883
Epoch 3/100
29/29 [==============================] - 42s 1s/step - loss: 0.0435 - accuracy: 0.9862 - val_loss: 1.0176 - val_accuracy: 0.8002
Epoch 4/100
29/29 [==============================] - 42s 1s/step - loss: 0.0305 - accuracy: 0.9892 - val_loss: 1.1257 - val_accuracy: 0.8002
Epoch 5/100
29/29 [==============================] - 42s 1s/step - loss: 0.0281 - accuracy: 0.9900 - val_loss: 1.0642 - val_accuracy: 0.8187
Epoch 6/100
29/29 [==============================] - 42s 1s/step - loss: 0.0251 - accuracy: 0.9919 - val_loss: 1.2394 - val_accuracy: 0.7872
Epoch 7/100
29/29 [==============================] - 43s 1s/step - loss: 0.0274 - accuracy: 0.9897 - val_loss: 1.0493 - val_accuracy: 0.8046
Epoch 8/100
2

# Save Model

In [0]:
%%time

# format : ai-kea-{day01}-{sey}

if train_model == True:
  history15000 = train(15000, 500)
  save_models("ai-kea", 15000)
else:
  generator, _, _ = load_models("ai-kea-day01-master",15000)

# Plot Learning Status

In [0]:
loss = history.history['loss']
epochs = range(1, len(loss)+1)

plt.figure(figsize=(10, 10))
plt.subplot(2, 1, 1)
plt.title('Validation Loss')
plt.semilogy(epochs, history.history['val_loss'], 'b', label='CNN')
plt.grid(True)
plt.xlabel('Epoch')
plt.ylabel('Loss')
#plt.ylim([0.0, 0.6])
plt.legend(loc='best')

plt.subplot(2, 1, 2)
plt.title('Validation Accuray')
plt.semilogy(epochs, history.history['val_accuracy'], 'b', label='CNN')
plt.grid(True)
plt.ylabel('Accuracy')
plt.ylim([0.5, 0.9])
plt.legend(loc='best')
plt.show()

In [0]:
print(np.max(history.history['val_accuracy']))